## XGBデフォルト

In [1]:
import numpy as np
import pandas as pd

import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, make_scorer, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split


In [3]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk1')

In [4]:
train_pkl.shape

(850, 11)

## 訓練データとテストデータに分割

In [5]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [6]:
# ターゲットと特徴量の分割
train_X = train.iloc[:, :-1]
train_y = train.disease.values

## 訓練データで訓練

In [7]:
XGB = XGBClassifier()

In [8]:
XGB = XGB.fit(train_X, train_y)

In [9]:
XGB.feature_importances_

array([0.02738552, 0.2993912 , 0.1011407 , 0.08259764, 0.09940092,
       0.17069837, 0.05222152, 0.04747922, 0.0813449 , 0.03834   ],
      dtype=float32)

In [10]:
sorted(
    zip(map(lambda x: round(x, 3), XGB.feature_importances_), train.iloc[:, :-1].columns),
    reverse=True)

[(0.299, 'T_Bil'),
 (0.171, 'AST_GOT'),
 (0.101, 'D_Bil'),
 (0.099, 'ALT_GPT'),
 (0.083, 'ALP'),
 (0.081, 'AG_ratio'),
 (0.052, 'TP'),
 (0.047, 'Alb'),
 (0.038, 'Gender_Male'),
 (0.027, 'Age')]

## テストデータで実行

In [11]:
# ターゲットと特徴量の分割
test_x = test.iloc[:, :-1]
test_y = test.disease.values

In [12]:
test_x.shape, test_y.shape

((170, 10), (170,))

In [13]:
pred_y = XGB.predict(test_x)

In [14]:
confusion_matrix(test_y, pred_y)

array([[87,  6],
       [16, 61]], dtype=int64)

In [15]:
accuracy_score(test_y, pred_y)

0.8705882352941177

## 検証データで実行

In [16]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk1')

In [17]:
valid.shape

(350, 10)

In [16]:
# ID の保存
# valid_pass = valid.id.values

In [18]:
# valid_X = valid.iloc[:, 1:]
valid_X = valid.copy()

In [19]:
valid_X.shape, train_X.shape

((350, 10), (680, 10))

In [20]:
pred_valid_y = XGB.predict(valid_X)

In [21]:
pred_valid_y.shape

(350,)

In [22]:
# type(valid_pass), type(pred_valid_y)

In [23]:
result_df = pd.DataFrame(pred_valid_y)

In [24]:
result_df.to_csv("./XGB_1.csv", header=False)